In [25]:
import pandas as pd
import numpy as np

from tqdm import tnrange

In [2]:
authors = pd.read_csv('authors.csv')
authors

,name,person_id
0,Everything+Is+Illuminated,145148
1,Robin+O%27Brien,297899
2,Nicholas+Gunn++(2012),250429
3,Aspasia+Stratigou,32765
4,Allison+Veltz,18689
...,...,...
560912,Sanaa+Kariakoo,544215
560913,Rock-a-teens,298403
560914,Jennifer+Lopez+Ft.+DJ+Mustard,450896
560915,Bobby+Sanabria+Conducting+The+Manhattan+School...,53831


In [3]:
tracks = pd.read_csv('tracks.csv')
tracks

,duration,playcount,name,artist_id,album_id,track_id
0,0,4,000003+Music+Instructor/_/Dj%27s+Rock+Da+House...,0,NaN,0
1,0,495,00-01/_/%D0%A2%D0%B5%D0%BA%D1%81%D1%82,1,NaN,1
2,0,2,0005.+Overkill/_/Overkill,2,NaN,2
3,0,2,000C+Tony+Dize/_/Ruleta+Rusa,3,NaN,3
4,0,1,000+Oscarcito/_/Tumbay%E2%80%9A+(Lyrics),4,NaN,4
...,...,...,...,...,...,...
3893285,144000,7,ZZ+Top/_/Hi+Fi+Mama,413280,NaN,3893299
3893286,191000,7,ZZ+Top/_/I+Thank+You,413280,NaN,3893300
3893287,169000,7,ZZ+Top/_/Low+Down+In+The+Street,413280,NaN,3893301
3893288,200000,7,ZZ+Top/_/Manic+Mechanic,413280,NaN,3893302


In [59]:
sessions_info = pd.read_csv('sessions.csv')
sessions_info

,id,numtracks,playtime,user_id
0,287144,23,4547,44361
1,287145,11,2907,44361
2,287146,16,3191,44361
3,287147,5,1162,44361
4,287140,2,250,42773
...,...,...,...,...
2764469,2480037,2,73,33058
2764470,2480030,1,-1,33058
2764471,540638,78,18220,24700
2764472,2480032,1,-1,33058


In [56]:
sessions = pd.read_csv('sessions_tracks.csv')
sessions

,id,playstart,playtime,playratio,action,session_id
0,4698874,0,58,NaN,play,287144
1,838286,58,310,1.01,play,287144
2,2588097,368,179,1.01,play,287144
3,2746740,547,342,1.44,play,287144
4,3873988,889,259,1.01,play,287144
...,...,...,...,...,...,...
26993351,3643664,17529,373,1.00,play,540638
26993352,2536532,17902,319,0.87,play,540638
26993353,512708,0,177,1.00,play,2480033
26993354,2672866,177,244,1.01,play,2480033


In [55]:
likes = pd.read_csv('loves.csv')
likes

,user_id,tracks_id
0,44542,2785601
1,44542,2785590
2,44542,143076
3,44542,143037
4,44542,143052
...,...,...
1692556,39896,3208469
1692557,39896,2756716
1692558,39896,3644495
1692559,39896,2247239


Создадим новый dataframe для истории прослушивания авторов с полями:

- всего треков
- всего альбомов
- всего лайков
- всего треков прослушали с повторами
- среднее время всех треков у автора
- средняя количество прослушиваний трека подряд (на повторе)


In [27]:
max_index_author = authors.sort_values(by='person_id')['person_id'].values[-1]

In [50]:
count_tracks_author = [0 for i in range(max_index_author)]
durations_author = [[] for i in range(max_index_author)]
albums_author = [[] for i in range(max_index_author)]

count_playcount_author = [0 for i in range(max_index_author)]

for i in tnrange(tracks.shape[0]):
    author_id = tracks['artist_id'][i]
    duration = tracks['duration'][i]
    album_id = tracks['album_id'][i]
    playcount = tracks['playcount'][i]
    
    if album_id is not None:
        albums_author[author_id].append(album_id)
    if duration > 0:
        durations_author[author_id].append(duration)
    
    count_tracks_author[author_id] += 1
    
    count_playcount_author[author_id] += playcount
    

In [38]:
def findMean(row):
    if len(row) == 0:
        return 0
    
    row = pd.Series(row)
    return row.mean()

In [101]:
%%time
durations_author_mean = pd.Series(durations_author).apply(findMean)
count_albums_author = list(map(lambda x: len(pd.Series(x).unique()), albums_author))


CPU times: user 2min 2s, sys: 1.4 s, total: 2min 3s
Wall time: 2min 3s


In [70]:
sessions_with_authors = sessions.merge(tracks, left_on='id', right_on='track_id')
sessions_with_authors

,id,playstart,playtime,playratio,action,session_id,duration,playcount,name,artist_id,album_id,track_id
0,838286,58,310,1.01,play,287144,307000,212,Dawes/_/When+My+Time+Comes,107103,48361.0,838286
1,838286,4037,307,1.00,play,982046,307000,212,Dawes/_/When+My+Time+Comes,107103,48361.0,838286
2,838286,13430,307,1.00,play,982049,307000,212,Dawes/_/When+My+Time+Comes,107103,48361.0,838286
3,838286,2941,310,1.01,play,961319,307000,212,Dawes/_/When+My+Time+Comes,107103,48361.0,838286
4,838286,10914,309,1.01,play,2497483,307000,212,Dawes/_/When+My+Time+Comes,107103,48361.0,838286
...,...,...,...,...,...,...,...,...,...,...,...,...
25909599,2564433,11290,210,1.04,play,540638,0,1384,Skarlett+Riot/_/Party+Hard,321698,NaN,2564433
25909600,2602587,12315,189,1.03,play,540638,0,202,Soldiers+of+a+Wrong+War/_/Walls,325968,NaN,2602587
25909601,720394,12504,212,NaN,play,540638,0,123,Crystalyne/_/Ghost+(Ft.+Martin+Broda),87396,NaN,720394
25909602,720401,12716,211,0.91,play,540638,0,29,Crystalyne/_/Tonight+I%27m+Getting+Over+You+(C...,87396,NaN,720401


In [73]:
ratio_author = [[] for i in range(max_index_author)]
real_count_tracks_author = [0 for i in range(max_index_author)]
playstart_mean_author = [[] for i in range(max_index_author)]

for i in tnrange(sessions_with_authors.shape[0]):
    author_id = sessions_with_authors['artist_id'][i]
    ratio = sessions_with_authors['playratio'][i]
    playstart = sessions_with_authors['playstart'][i]
    
    if ratio is not None:
        ratio_author[author_id].append(ratio)
    
    real_count_tracks_author[author_id] += 1
    
    playstart_mean_author[author_id].append(playstart)
    

In [88]:
likes_tracks = likes.merge(tracks, right_on='track_id', left_on='tracks_id')
likes_tracks

,user_id,tracks_id,duration,playcount,name,artist_id,album_id,track_id
0,44542,2785601,208000,1300,The+Amphetameanies/_/This+Boy,349295,160894.0,2785601
1,44542,2785590,0,1203,The+Amphetameanies/_/Backbeat+Fucker,349295,NaN,2785590
2,44542,143076,0,16806,Allo+Darlin%27/_/We+Come+From+The+Same+Place,18754,NaN,143076
3,13516,143076,0,16806,Allo+Darlin%27/_/We+Come+From+The+Same+Place,18754,NaN,143076
4,44542,143037,212000,2331,Allo+Darlin%27/_/Bright+Eyes,18754,7830.0,143037
...,...,...,...,...,...,...,...,...
1651777,39880,2514365,230000,130,Shakespeare%27s+Sister/_/Stay+(Radio+Mix),315167,NaN,2514365
1651778,39880,2254906,354000,1250,Project+Pitchfork/_/If+I+Could+(First+Dream),281944,129870.0,2254906
1651779,39880,263514,0,344,Astairre/_/Ein+Hunger+der+mich+frisst,32971,NaN,263514
1651780,39880,3194439,459000,398,VNV+Nation/_/Still+Waters+(unreleased),395571,NaN,3194439


In [90]:
likes_author = [0 for i in range(max_index_author)]

for i in tnrange(likes_tracks.shape[0]):
    author_id = likes_tracks['artist_id'][i]
    
    likes_author[author_id] += 1
    

In [78]:
%%time
ratio_author_mean = pd.Series(ratio_author).apply(findMean)
playstart_author_mean = pd.Series(playstart_mean_author).apply(findMean)

CPU times: user 2min 8s, sys: 608 ms, total: 2min 8s
Wall time: 2min 9s


In [112]:
ready_data_authors_history = pd.DataFrame({'ratio': ratio_author_mean, 'playstart': playstart_author_mean, 'likes':likes_author, 'duration': durations_author_mean, 'albums': count_albums_author, 'real_count_tracks': real_count_tracks_author, 'count_tracks': count_tracks_author})
ready_data_authors_history = ready_data_authors_history[ready_data_authors_history['real_count_tracks'] != 0]
ready_data_authors_history

,ratio,playstart,likes,duration,albums,real_count_tracks,count_tracks
1,1.358182,6180.363636,0,0.0,1,11,1
2,1.080000,3592.000000,0,0.0,1,1,1
3,1.485000,2187.000000,0,0.0,1,2,1
5,1.092500,28097.750000,1,0.0,1,4,2
6,1.020000,1692.000000,0,0.0,1,1,1
...,...,...,...,...,...,...,...
473975,0.980000,5941.000000,0,299000.0,1,1,1
473976,0.975000,6490.333333,0,0.0,1,6,4
473977,1.000000,8177.000000,0,0.0,1,1,1
473978,1.165577,5232.096154,1,218300.0,1,52,12


In [113]:
ready_data_authors_history = ready_data_authors_history.reset_index()
ready_data_authors_history.columns = ['author_id', 'ratio', 'playstart', 'likes', 'duration', 'albums', 'real_count_tracks', 'count_tracks']
ready_data_authors_history

,author_id,ratio,playstart,likes,duration,albums,real_count_tracks,count_tracks
0,1,1.358182,6180.363636,0,0.0,1,11,1
1,2,1.080000,3592.000000,0,0.0,1,1,1
2,3,1.485000,2187.000000,0,0.0,1,2,1
3,5,1.092500,28097.750000,1,0.0,1,4,2
4,6,1.020000,1692.000000,0,0.0,1,1,1
...,...,...,...,...,...,...,...,...
437089,473975,0.980000,5941.000000,0,299000.0,1,1,1
437090,473976,0.975000,6490.333333,0,0.0,1,6,4
437091,473977,1.000000,8177.000000,0,0.0,1,1,1
437092,473978,1.165577,5232.096154,1,218300.0,1,52,12


,author_id,ratio,playstart,likes,duration,albums,real_count_tracks,count_tracks
0,1,1.358182,6180.363636,0,0.0,1,11,1
1,2,1.080000,3592.000000,0,0.0,1,1,1
2,3,1.485000,2187.000000,0,0.0,1,2,1
3,5,1.092500,28097.750000,1,0.0,1,4,2
4,6,1.020000,1692.000000,0,0.0,1,1,1
...,...,...,...,...,...,...,...,...
437089,473975,0.980000,5941.000000,0,299000.0,1,1,1
437090,473976,0.975000,6490.333333,0,0.0,1,6,4
437091,473977,1.000000,8177.000000,0,0.0,1,1,1
437092,473978,1.165577,5232.096154,1,218300.0,1,52,12


In [117]:
ready_data_authors_history = ready_data_authors_history[ready_data_authors_history['ratio'] >= 0]
ready_data_authors_history

,author_id,ratio,playstart,likes,duration,albums,real_count_tracks,count_tracks
0,1,1.358182,6180.363636,0,0.0,1,11,1
1,2,1.080000,3592.000000,0,0.0,1,1,1
2,3,1.485000,2187.000000,0,0.0,1,2,1
3,5,1.092500,28097.750000,1,0.0,1,4,2
4,6,1.020000,1692.000000,0,0.0,1,1,1
...,...,...,...,...,...,...,...,...
437089,473975,0.980000,5941.000000,0,299000.0,1,1,1
437090,473976,0.975000,6490.333333,0,0.0,1,6,4
437091,473977,1.000000,8177.000000,0,0.0,1,1,1
437092,473978,1.165577,5232.096154,1,218300.0,1,52,12


In [118]:
ready_data_authors_history.to_csv('prepared_authors_history.csv', index=False)